In [ ]:
import pandas as pd
import os

df_easily = pd.read_csv("C:/Users/benysar/Desktop/easily.csv", delimiter='|', encoding="utf-8") #CHANGER PATH
df_warehouse = pd.read_csv("C:/Users/benysar/Desktop/warehouse.csv", delimiter='|', encoding="utf-8") #CHANGER PATH

## Suppression des doublons parfaits

In [ ]:
df_easily_nodups = df_easily.drop_duplicates()
df_warehouse_nodups = df_easily.drop_duplicates()

## Pre-processing des dates et des nans

In [ ]:
# Conversion des nan en float
df_easily_nodups['pat_ipp'] = df_easily_nodups['pat_ipp'].astype(float)
df_warehouse_nodups['pat_ipp'] = df_warehouse_nodups['pat_ipp'].astype(float)

In [ ]:
# Pour les dates au format 'YYYY-MM-DD HH:MM:SS', couper la chaîne aux 10 premiers caractères 
df_warehouse_nodups['DOCUMENT_DATE'] = df_warehouse_nodups['DOCUMENT_DATE'].str.slice(0, 10)
df_easily_nodups['DOCUMENT_DATE'] = df_easily_nodups['DOCUMENT_DATE'].str.slice(0, 10)

## Création de la date, et mergre pour obtenir les différences

In [ ]:
# Étape 1: Créer une nouvelle colonne pour la date du document
df_easily_nodups['DOCUMENT_DATE_EASILY'] = df_easily_nodups.apply(
    lambda row: row['rea'] if pd.notnull(row['rea']) else (row['crea'] if pd.notnull(row['crea']) else row['modif']), axis=1)

df_warehouse_nodups['DOCUMENT_DATE_WAREHOUSE'] = df_warehouse_nodups['DOCUMENT_DATE']

# Étape 2: Fusionner les DataFrames sur les ID patients et les noms des documents
merged_df = pd.merge(df_easily_nodups, df_warehouse_nodups, left_on=['pat_ipp', 'doc_nom', 'DOCUMENT_DATE_EASILY'], 
                     right_on=['HOSPITAL_PATIENT_ID', 'TITLE', 'DOCUMENT_DATE_WAREHOUSE'], how='outer', indicator=True)

# Étape 3: Filtrer les résultats pour obtenir les différences
differences = merged_df[merged_df['_merge'] != 'both']

## Affichage des résultats

In [ ]:
# Calculer la répartition des valeurs de fusion
merge_counts = merged_df['_merge'].value_counts()

# Renommer les valeurs
merge_counts = merge_counts.rename(index={
    'left_only': 'Présent seulement dans df_easily',
    'right_only': 'Présent seulement dans df_warehouse',
    'both': 'Présent dans les deux DataFrames'
})

# Afficher les résultats
print(merge_counts)


In [ ]:
merged_df['DOCUMENT_DATE_EASILY_year'] = merged_df['DOCUMENT_DATE_EASILY'].str.slice(0, 3)

merged_df_2023 = merged_df[merged_df['DOCUMENT_DATE_EASILY_year'] == '2023']
merge_counts_2023 = merged_df_2023['_merge'].value_counts()

# Renommer les valeurs
merge_counts_2023 = merge_counts_2023.rename(index={
    'left_only': 'Présent seulement dans df_easily en 2023',
    'right_only': 'Présent seulement dans df_warehouse en 2023',
    'both': 'Présent dans les deux DataFrames en 2023'
})

# Afficher les résultats
print(merge_counts_2023)

## Détail des documents manquants pour chaque connecteurs

In [ ]:
# Perform the merge
df = pd.merge(df_easily_nodups, df_warehouse_nodups, 
                     left_on=['pat_ipp', 'doc_nom', 'DOCUMENT_DATE_EASILY'], 
                     right_on=['HOSPITAL_PATIENT_ID', 'TITLE', 'DOCUMENT_DATE_WAREHOUSE'], 
                     how='outer', indicator=True)

df = df[df['_merge'] != 'both']

In [ ]:

# Filter rows where 'DOCUMENT_DATE_EASILY_year' is '2023'
df['DOCUMENT_DATE_EASILY_year'] = df['DOCUMENT_DATE_EASILY'].str.slice(0, 3)
filtered_df = df[df['DOCUMENT_DATE_EASILY_year'] == '2023']

def save_app_nom_data_to_excel(df):
    # Base directory where files will be saved
    base_dir = "C:/Users/benysar/testing_data/doc_manquants_warehouse"  #CHANGER PATH

    # Iterate through unique values in 'app_nom'
    for app_name in df['app_nom'].unique():
        # Sanitize the app_name to remove or replace invalid characters for file names
        sanitized_app_name = app_name.replace("/", "_").replace("\\", "_").replace(" ", "_").lower()

        # Filter the DataFrame for the current app_name
        filtered_df = df[df['app_nom'] == app_name]

        filtered_df['pat_ipp'] = filtered_df['pat_ipp'].astype(int)

        filtered_df = filtered_df[['doc_nom', 'pat_ipp', 'app_nom', 'DOCUMENT_DATE_EASILY']]

        # File name based on sanitized app_name
        file_name = f"{base_dir}/{sanitized_app_name}_2023.xlsx"

        # Save the DataFrame to an Excel file
        filtered_df.to_excel(file_name, index=False)
        print(f"Data saved to {file_name}")

# Example usage
save_app_nom_data_to_excel(filtered_df)


In [ ]:
# Get counts and normalized counts
counts = filtered_df["app_nom"].value_counts()
normalized_counts = filtered_df["app_nom"].value_counts(normalize=True)

# Create a new DataFrame with these values
df_counts = pd.DataFrame({
    'Counts': counts,
    'Normalized Counts': normalized_counts
})

print(df_counts)

# Correcting the file path with the proper extension
file_path = "C:/Users/benysar/testing_data/doc_manquants_warehouse/RESUME_MANQUANTS.xlsx" #CHANGER PATH

# Saving the DataFrame to an Excel file
df_counts.to_excel(file_path, index=False)
